# Tree based methods for prediction

This tutorial is meant to be an introduction to tree based methods for prediction. We start with the most basic model, a decision tree, and work our way up to the more recent work on gradient boosting. We will be alternating between sample datasets available with sci-kit learn and a medical dataset made available in this repository.

In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn import datasets
import pydotplus

import matplotlib.pyplot as plt

# used to display trees
from IPython.display import Image

%matplotlib inline

In [ ]:
def plot_model_pred_2d(mdl, X, y, feat):
    # look at the regions in a 2d plot
    # based on scikit-learn tutorial plot_iris.html

    # get minimum and maximum values
    x0_min = X[:, 0].min()
    x0_max = X[:, 0].max()
    x1_min = X[:, 1].min()
    x1_max = X[:, 1].max()

    xx, yy = np.meshgrid(np.linspace(x0_min, x0_max, 100),
                         np.linspace(x1_min, x1_max, 100))

    Z = mdl.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # plot the contour - colouring different regions
    cs = plt.contourf(xx, yy, Z, cmap='Blues')

    # plot the individual data points - colouring by the *true* outcome
    color = y.ravel()
    plt.scatter(X[:, 0], X[:, 1], c=color, marker='o', s=40, cmap='Blues')

    plt.xlabel(feat[0])
    plt.ylabel(feat[1])
    plt.axis("tight")
    plt.colorbar()

# Dataset

The dataset we'll use is the UCI Breast Cancer classification dataset.

In [ ]:
# real example
df = datasets.load_iris()

# if you want a description of the dataset, uncomment the below line
print(df['DESCR'])

idx = [0,2]
X = df['data'][50:,idx]
y = df['target'][50:]

feat = [df['feature_names'][x] for x in idx]

# Decision trees

There are a few flavours - but we will be using Classification and Regression Trees (CART) - well described in a book of the same name [1]. This section will go over an two example datasets: one for classification and one for regression.

* build a single tree for each
* examine how the tree partitions the space
* explain how this works in terms of the cost function split on at each node

[1] L. Breiman, J. Friedman, R. Olshen, and C. Stone. Classification and Regression Trees. Wadsworth, Belmont, CA, 1984.

In [ ]:
mdl = tree.DecisionTreeClassifier(criterion='entropy', splitter='best')
mdl = mdl.fit(X,y)

In [ ]:
# examine the tree
tree.export_graphviz(mdl, out_file='tree.dot',
                         feature_names=feat, 
                         filled=True, rounded=True)  
graph = pydotplus.graphviz.graph_from_dot_file("tree.dot") 
Image(graph.create_png())  

In [ ]:
# look at the regions in a 2d plot
# based on scikit-learn tutorial plot_iris.html
plt.figure(figsize=[16,9])
plot_model_pred_2d(mdl, X, y, feat)
plt.show()

* above model is complicated
* likely "overfit"

# Boosting

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=1)
mdl = ensemble.AdaBoostClassifier(base_estimator=clf,n_estimators=5)
mdl = mdl.fit(X,y)


# plot the final prediction
plt.figure(figsize=[16,9])
plot_model_pred_2d(mdl, X, y, feat)
plt.show()

In [ ]:
# break it down into 5 subfigures

print('Each estimator, one at a time...')
fig = plt.figure(figsize=[16,9])
for i, estimator in enumerate(mdl.estimators_):
    ax = fig.add_subplot(2,3,i+1)
    plot_model_pred_2d(estimator, X, feat)
    plt.title('Estimator {}'.format(i+1),fontsize=16)
plt.show()


# plot the final prediction
plt.figure(figsize=[16,9])
plot_model_pred_2d(mdl, X, y, feat)
plt.show()

In [ ]:
# break it down into 5 subfigures
clf = tree.DecisionTreeClassifier(max_depth=3)
mdl = ensemble.AdaBoostClassifier(base_estimator=clf,n_estimators=5)
mdl = mdl.fit(X,y)

print('Each estimator, one at a time...')
fig = plt.figure(figsize=[16,9])
for i, estimator in enumerate(mdl.estimators_):
    ax = fig.add_subplot(2,3,i+1)
    plot_model_pred_2d(estimator, X, y, feat)
    plt.title('Estimator {}'.format(i+1),fontsize=16)
plt.show()


# plot the final prediction
plt.figure(figsize=[16,9])
plot_model_pred_2d(mdl, X, y, feat)
plt.show()

# Bagging

Bagging is a form of *ensemble learning*: we aim to build a single good model by combining many models together. For example, we could train 10 decision trees, and then combine the predictions of those 10 decision trees (say by majority vote or averaging) in order to get a better answer.

It's not quite that simple though. In the above CART, recall that the training method involved (i) determining the optimal split point across all features, (ii) splitting the data into two groups, and (iii) repeating this process until completion. If we gave the decision tree the same dataset, we would always get the same tree built - which means there is absolutely no point in repeatedly building trees. Ideally, we would be able to acquire a new dataset each time we decide to build a new tree. Practically that's usually not possible, but happily we can take advantage of a magic trick from statistics: the bootstrap.

The bootstrap is a very popular method, and loosely speaking it allows one to repeatedly generate "new" datasets from the current dataset. So our new methodology is:

1. Generate a bootstrap sample of the dataset
2. Build a decision tree
3. Repeat 1-2 until we're happy with the overall model

And that's it! Let's try it with the housing dataset.

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=1)
mdl = ensemble.BaggingClassifier(base_estimator=clf, n_estimators=5)
mdl = mdl.fit(X,y)



print('Each estimator, one at a time...')
fig = plt.figure(figsize=[16,9])
for i, estimator in enumerate(mdl.estimators_):
    ax = fig.add_subplot(2,3,i+1)
    plot_model_pred_2d(estimator, X, y, feat)
    plt.title('Estimator {}'.format(i+1),fontsize=16)
plt.show()


# plot the final prediction
plt.figure(figsize=[16,9])
plot_model_pred_2d(mdl, X, y, feat)
plt.show()

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=1)
mdl = ensemble.BaggingClassifier(base_estimator=clf, n_estimators=10)
mdl = mdl.fit(X,y)



print('Each estimator, one at a time...')
fig = plt.figure(figsize=[16,9])
for i, estimator in enumerate(mdl.estimators_):
    ax = fig.add_subplot(3,4,i+1)
    plot_model_pred_2d(estimator, X, y, feat)
plt.show()


# plot the final prediction
plt.figure(figsize=[16,9])
plot_model_pred_2d(mdl, X, y, feat)
plt.show()

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=None)
mdl = ensemble.BaggingClassifier(base_estimator=clf, n_estimators=10)
mdl = mdl.fit(X,y)



print('Each estimator, one at a time...')
fig = plt.figure(figsize=[16,9])
for i, estimator in enumerate(mdl.estimators_):
    ax = fig.add_subplot(3,4,i+1)
    plot_model_pred_2d(estimator, X, y, feat)
plt.show()


# plot the final prediction
plt.figure(figsize=[16,9])
plot_model_pred_2d(mdl, X, y, feat)
plt.show()

# Random Forest

Above, we used the bootstrap to randomly resample our data to generate "new" datasets to build trees from. The Random Forest takes this one step further: instead of just resampling our data, we also select only a fraction of the features to include. It turns out that this extra randomization tends to improve the performance of our models.

Let's try and train the model now:

In [ ]:
mdl = ensemble.RandomForestClassifier(n_estimators=10)
mdl = mdl.fit(X,y)

plt.figure(figsize=[16,9])
plot_model_pred_2d(mdl, X, y, feat)
plt.show()

# Gradient Boosting

From scikit manual

The advantages of GBRT are:
* Natural handling of data of mixed type (= heterogeneous features)
* Predictive power
* Robustness to outliers in output space (via robust loss functions)

The disadvantages of GBRT are:
* Scalability, due to the sequential nature of boosting it can hardly be parallelized.

In [ ]:
mdl = ensemble.GradientBoostingClassifier(n_estimators=10)
mdl = mdl.fit(X, y)


plt.figure(figsize=[16,9])
plot_model_pred_2d(mdl, X, y, feat)
plt.show()

# Running through a slightly harder dataset

Breast cancer dataset.

In [ ]:
# real example
df = datasets.load_breast_cancer()

# if you want a description of the dataset, uncomment the below line
print(df['DESCR'])

# pick index of the features to use (only pick 2)
#    :Attribute Information (in order):
#        0 - CRIM     per capita crime rate by town
#        1 - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
#        2 - INDUS    proportion of non-retail business acres per town
#        3 - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
#        4 - NOX      nitric oxides concentration (parts per 10 million)
#        5 - RM       average number of rooms per dwelling
#        6 - AGE      proportion of owner-occupied units built prior to 1940
#        7 - DIS      weighted distances to five Boston employment centres
#        8 - RAD      index of accessibility to radial highways
#        9 - TAX      full-value property-tax rate per $10,000
#       10 - PTRATIO  pupil-teacher ratio by town
#       11 - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
#       12 - LSTAT    % lower status of the population
#       13 - MEDV     Median value of owner-occupied homes in $1000's


idx = [1,29]
X = df['data'][:,idx]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.30, random_state=42)

feat = [x for x in df['feature_names'][idx]]

In [ ]:
clf = dict()
#clf['Decision Tree'] = tree.DecisionTreeClassifier(criterion='entropy', splitter='best').fit(X,y)
clf['Gradient Boosting'] = GradientBoostingClassifier(n_estimators=10).fit(X_train, y_train)
clf['Random Forest'] = ensemble.RandomForestClassifier(n_estimators=10).fit(X_train, y_train)
clf['Bagging'] =  ensemble.BaggingClassifier(n_estimators=10).fit(X_train, y_train)
clf['AdaBoost'] =  ensemble.AdaBoostClassifier(n_estimators=10).fit(X_train, y_train)

fig = plt.figure(figsize=[16,9])
for i, curr_mdl in enumerate(clf):
    score = clf[curr_mdl].score(X_test, y_test)
    print('{}\t{}'.format(score, curr_mdl))
    
    ax = fig.add_subplot(2,2,i+1)
    plot_model_pred_2d(clf[curr_mdl], X_test, y_test, feat)
    
    plt.title(curr_mdl)
    
plt.show()


In [ ]:
# even better, use cross-validation
for curr_mdl in clf:
    scores = model_selection.cross_val_score(clf[curr_mdl], X, y, cv=5, scoring='accuracy')
    auc = model_selection.cross_val_score(clf[curr_mdl], X, y, cv=5, scoring='roc_auc')
    print('{:0.3f}\t{:0.3f}\t{}'.format(scores.mean(), auc.mean(), curr_mdl))

# Exercise

We'll now practice using these models on a dataset acquired from patients admitted to intensive care units at the Beth Israel Deaconness Medical Center in Boston, MA. All patients in the cohort stayed for at least 48 hours, and the goal of the prediction task is to predict in-hospital mortality. If you're interested, you can read more about the dataset here: http://physionet.org/challenge/2012/

The following command will download a "preprocessed" version of the dataset to the current directory. The data is originally provided as hourly observations for a number of variables, and the preprocessing step involved extracting summary statistics across all these observations.

Let's look at the first five features to get an idea of what this dataset looks like:

* build a single tree for each
* examine how the tree partitions the space
* explain how this works in terms of the cost function split on at each node

End of section exercise: build a decision tree using PhysionetChallenge2012_train.csv